# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-1. Применяем простейший локальный поиск.

В этой задаче Вам предлагается попробовать стандартную технику локального поиска (local search) в применении к задаче о сбалансированном разбиении графа. Мы будем рассматривать безвесовый вариант задачи с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* [Скачайте](http://mat.gsia.cmu.edu/COLOR/instances.html#XXMYC) файлы mycielX.col  (`for X in range(1,8)`).  (Если интересно, откуда такие графы берутся и чем интересны, см. конструкцию Зыкова—Мыцельского [здесь](https://docs.com/dainiak/3327).)
* Для каждого из графов найдите локальным поиском локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Ваша функция `basic_local_search` должна принимать на вход граф в формате, предоставляемом функцией `read_col_file`, и возвращать найденное локально минимальное разбиение просто как множество либо список вершин, лежащих в одной любой из двух компонент разбиения.

In [33]:
import random
from tqdm import tqdm

In [49]:
def read_col_file(filename: str) -> tuple:
    with open(filename, 'r') as file:
        vertices, edges = set(), set()
        for line in file:
            line = line.strip()
            if line.startswith('p'):
                vertices = set(range(1, int(line.split()[-2]) + 1))
            elif line.startswith('e'):
                edges.add(tuple(map(int, line.split()[-2:])))
        return (vertices, edges)

In [50]:
def cost_cut(left_set: set, right_set: set, edges:set) -> int:
        cost = 0
        for edge in edges:
            vertex1, vertex2 = edge
            if (vertex1 in left_set and vertex2 in right_set) or (vertex1 in right_set and vertex2 in left_set):
                cost += 1
        return cost

In [51]:
def get_partition(vertexes):
    left = set(random.sample(vertexes, len(vertexes) // 2))
    right = vertexes - left 
    return left, right

In [72]:
def basic_local_search(graph: tuple) -> tuple:
    vertices, edges = graph
    n = len(vertices)
    left, right = random_partition(vertices)
    cur_cut_cost = cut_cost(left, right, edges)
    while True:
        min_new_left, min_new_right, min_new_cut_cost = [], [], cur_cut_cost
        for vertex1 in left:
            for vertex2 in right:
                new_left, new_right = left.copy(), right.copy()
                new_left.remove(vertex1)
                new_right.remove(vertex2)
                new_left.add(vertex2)
                new_right.add(vertex1)  
                cur_cut_cost = cut_cost(new_left, new_right, edges)
                if cur_cut_cost < min_new_cut_cost:
                    min_new_left, min_new_right, min_new_cut_cost = new_left, new_right, cur_cut_cost
        if min_new_left != []:
            left, right = min_new_left, min_new_right
            if (len(left) > len(right)):
                left, right = right, left
            cur_cut_cost = min_new_cut_cost
        else:
            return (left, right)

In [80]:
for i in tqdm_notebook(range(3, 8)):
    vertices, edges = read_col_file('myciel%d.col'%(i))
    left, right = basic_local_search((vertices, edges))
    cut_cost_value = cut_cost(left, right, edges)
    print('myciel%d.col'%(i))
    print(' left =', left, '\n', 'right =', right, '\n', "cut cost = ", cut_cost_value, '\n')

myciel3.col
 left = {1, 3, 5, 7, 9} 
 right = {2, 4, 6, 8, 10, 11} 
 cut cost =  8 

myciel4.col
 left = {2, 3, 6, 8, 13, 16, 17, 19, 21, 22, 23} 
 right = {1, 4, 5, 7, 9, 10, 11, 12, 14, 15, 18, 20} 
 cut cost =  29 

myciel5.col
 left = {1, 2, 3, 4, 5, 7, 9, 10, 12, 14, 17, 18, 21, 24, 25, 26, 27, 30, 34, 35, 37, 38, 39} 
 right = {6, 8, 11, 13, 15, 16, 19, 20, 22, 23, 28, 29, 31, 32, 33, 36, 40, 41, 42, 43, 44, 45, 46, 47} 
 cut cost =  96 

myciel6.col

 left = {5, 7, 8, 9, 10, 11, 13, 15, 16, 18, 19, 20, 21, 22, 24, 26, 28, 29, 30, 31, 32, 33, 34, 43, 46, 48, 50, 52, 55, 56, 58, 60, 61, 62, 63, 65, 66, 67, 69, 71, 73, 76, 78, 79, 81, 92, 93} 
 right = {1, 2, 3, 4, 6, 12, 14, 17, 23, 25, 27, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 47, 49, 51, 53, 54, 57, 59, 64, 68, 70, 72, 74, 75, 77, 80, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 94, 95} 
 cut cost =  297 

myciel7.col
 left = {1, 3, 12, 13, 14, 15, 17, 18, 19, 20, 21, 24, 25, 28, 29, 30, 33, 34, 35, 36, 39, 40, 41, 42, 43, 44, 46, 